In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import mysql.connector
import pandas as pd
import json

In [3]:
def open_database():
    con = mysql.connector.connect(user='dbadmin', password='washywashy',host='127.0.0.1',database='tests')
    return con, con.cursor()

def read_csv(filename):
    df = pd.read_csv(filename, low_memory=False)
    return df

def select_columns(df, columns):
    return df[columns]

def get_data():
    df = read_csv('./data/movies_imbd.csv')
    return df

In [4]:
def get_column_df(col):
    df = read_csv('./data/movies_imbd.csv')
    df = select_columns(df,[col])
    df = df[~pd.isnull(df[col])]
    df.fillna("",inplace=True)
    return df

In [5]:
def get_production_companies_data():
    production_companies = {}
    df = get_column_df('production_companies')
    for c in df.sample(1000).values:
        if(c[0] != '[]'):
            parts = c[0].split('},')        
            for p in parts:
                p = p.replace("[{'id","{'id").replace(" {'id'", "{'id'").replace("}]", "}").replace("} ", "}")
                p = p.replace("[{'name","{'name")
                if p[-1] != '}':
                    p = p + "}"
                    p = p.replace("{'name': ", "").replace("'id': ", "").replace('}', '')        
                    parts2 = p.split("', ")
                    if len(parts2) == 2:
                        name = parts2[0].strip()[1:]
                        id = parts2[1]
                        production_companies[id] = name
                    else:
                        parts2 = p.split('", ')
                        name = parts2[0].strip()[1:]
                        id = parts2[1]
                        production_companies[id] = name
    return production_companies


In [6]:
def from_key_to_key(s, fk, tk):
    fp = s.find("'"++"':")
    tp = s.find("'"+tk+"':")    
    return s[fp+len(fk)+4:tp-2]

def get_single_id_name(s):
    return from_key_to_key(s, "id", "name"), from_key_to_key(s, "name", "poster_path")

def get_columns_json():
    df = get_column_df('genres')
    for s in df.values:
            if (s[0] != '[]'):
                parts = s[0].split('},')        
                for p in parts:
                    p = p.replace("[{'id","{'id").replace(" {'id'", "{'id'").replace("}]", "}").replace("} ", "}")
                    p = p.replace("[{'name","{'name")
                    if p[-1] != '}':
                        p = p + "}"
                    p = p.replace("'", '"')
                    obj = json.loads(p)
                    print(obj)
                    return(obj)
           

In [7]:
def test_for_single_id_name():
    df = get_column_df("belongs_to_collection")
    i = 0
    for s in df.values:
        id, name = get_single_id_name(s[0])
        print(i, "id => " + id, ",  name => " + name)
        i += 1        

In [8]:
#test_for_single_id_name()
# get_genres_json()

In [9]:
def create_production_companies_table():
    con, cursor = open_database()
    cursor.execute("drop table if exists tests.production_companies")
    create_table_sql = """
    create table tests.production_companies (
        id int,
        name varchar(512),
        primary key(id)
    )
    """
    cursor.execute(create_table_sql)
    con.commit()    
    con.close()
    
def insert_production_companies_data():
    pc = get_production_companies_data()
    values = []
    for key in pc:
        values.append((key, pc[key]))
    con, cursor = open_database()
    cursor.executemany("insert into production_companies (id, name) values (%s, %s)", values)
    con.commit()    
    con.close()    

In [10]:
# create_production_companies_table()
# insert_production_companies_data()

In [11]:
def get_production_countries():
    df = get_column_df('production_countries')
    production_countries = {}
    for c in df.values:
        if (c[0] != '[]'):
            lines = c[0].split('},')
            for l in lines:
                l = l.replace("[{'iso_3166_1","{'iso_3166_1").replace("}]","}")

                if l[-1] != "}":
                    l = l + "}"
                l = l.strip()

                initials = from_key_to_key(l,"iso_3166_1", "name" )
                initials = initials.replace("'", "")

    #           getting country was different
                fp = l.find("'"+'name'+"':")
                tp = l.find(""+'}'+"")
                country = l[fp+len('name')+5:tp-1]

                production_countries[initials] = country
    return production_countries